In [1]:
# Required libraries
import tempfile, os

import numpy as np
import pandas as pd
import geopandas as gpd
from tqdm import tqdm

import warnings

import rasterio
import xarray as xr
import rioxarray as rxr

import requests
import json

import contextily as ctx
# from contextily import Place

# Addresses SSL error when interacting with worldpop data
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
# Common directories
DATA_DIR = "X:/Work/GeoTemp/CCDR/Nepal/data_script/"
ADM_bound = "X:/Work/GeoTemp/CCDR/Nepal/data_script/adm"

In [3]:
def damage_factor(x):
    """A polynomial fit to average damage across multiple sectors relative 
    to water depth in meters in Asia.

    The sectors are commercial, industry, transport, agriculture, infrastructure and residential.

    Values are capped between 0 and 1, where values >= 6m = 1

    References
    ----------
    .. [1] JRC, 2017
    """
    return np.maximum(0.0, np.minimum(1.0, 0.00723*x**3 - 0.1*x**2 + 0.506*x))


In [4]:
# Stub for user input
# TODO: Make this nicer and more accessible for users

country = "NPL"
exp_cat = ["population", "land cover"]
time_horizon = [2050, 2080]
rcp_scenario = ["2.6", "4.5", "6.5", "8.5"]

# Settings
agg_criteria = "mean"  # ["max", "mean"]
class_range = range(3, 11)  # remember that python uses end-exclusive range, so this is 3-10

selected_bin_edges = [0.5, 1, 1.5, 2, 2.5, 3]
min_haz_threshold = np.min(selected_bin_edges)  # determine min/max values from user-selected edges
max_haz_threshold = np.max(selected_bin_edges)
selected_bin_edges += [np.inf] # add inf last to cover anything above max threshold.

num_bins = len(selected_bin_edges)-1  # default number of bins, within the range of `class_range`

In [5]:
# Testing data file locations
# TODO: Temp data store, to be replaced with a config spec (.env file?) before deployment

# pop_fn = f"{DATA_DIR}/cache/{fid}_{cache_fn}"
pop_fn = f"{DATA_DIR}/cache/WorldPop20_{country}_ppp_UNadj_constrained.tif"

# Flood data location (TODO: replace with pointer to downloaded data store)
flood_RP_data_loc = f"X:/Work/GeoTemp/CCDR/Nepal/data_script/"

In [6]:
country_code_map = {
    "NPL": 175
}

In [11]:
# Load or save ISO3 country list
iso3_path = f"{DATA_DIR}cache/iso3.json"
if not os.path.exists(iso3_path):
    resp = json.loads(requests.get(f"https://www.worldpop.org/rest/data/pop/wpgp?iso3={country}").text)

    with open(iso3_path, 'w') as outfile:
        json.dump(resp, outfile)
else:
    with open(iso3_path, 'r') as infile:
        resp = json.load(infile)


# TODO: User to select population data set
# Target population data files are extracted from the JSON list downloaded above
metadata = resp['data'][1]
data_src = metadata['files']

In [12]:
# Save population data to cache location
for data_fn in tqdm(data_src):
    fid = metadata['id']
    cache_fn = os.path.basename(data_fn)

    # Look for indicated file in cache directory
    # Use the data file if it is found, but warn the user. 
    # (if data is incorrect or corrupted, they should delete it from cache)
    if f"{fid}_{cache_fn}" in os.listdir(f"{DATA_DIR}/cache"):
        warnings.warn(f"Found {fid}_{cache_fn} in cache, skipping...")
        continue

    # Write to cache file if not found
    with open(f"{DATA_DIR}/cache/{fid}_{cache_fn}", "wb") as handle:
        response = requests.get(data_fn)
        handle.write(response.content)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:27<00:00, 27.20s/it]


Load country boundaries from ADM geopackage file which includes ISO3 code related to country name

In [14]:
country_bounds = gpd.read_file(os.path.join(ADM_bound, "NPL_ADM.gpkg"))

In [15]:
valid_RPs = [10, 100, 1000]

# Open population dataset
pop_data = rxr.open_rasterio(pop_fn)

# Indicate 0 values as representing no data.
pop_data.rio.write_nodata(0, inplace=True)

# Get raw array for population data
pop_array = pop_data[0].values

# Prep result structure
adm_dataset = gpd.read_file(os.path.join(ADM_bound, "NPL_ADM.gpkg"), layer="ADM3")
adm2_data = adm_dataset.loc[adm_dataset.ADM0_CODE == country_code_map[country], :]

# Create dataframe to hold results
num_rows = len(adm2_data.index)
adm_details = {
    "ADM3_CODE": adm2_data.LOCAL,
    "ADM3_NAME": adm2_data.LOCAL
}

result_df = pd.DataFrame(adm_details)
pop_sum_cols = [f"RP{rp_i}_pop_sum" for rp_i in valid_RPs]
EAI_cols = [f"RP{rp_i}_EAI" for rp_i in valid_RPs]

num_new_cols = len(pop_sum_cols + EAI_cols)
result_df.loc[:, pop_sum_cols + EAI_cols] = np.array([np.zeros(num_rows), ] * num_new_cols).T


for rp in valid_RPs:

    # Load ADM2 based on country code value
    adm3_data = adm_dataset.loc[adm_dataset.ADM0_CODE == country_code_map[country], :]

    # Load corresponding flood dataset
    flood_data = rxr.open_rasterio(flood_RP_data_loc+f"RP{rp}.tif")

    # Reproject and clip raster to same bounds as population data
    flood_data = flood_data.rio.reproject_match(pop_data)
    flood_data = flood_data.rio.clip_box(*pop_data.rio.bounds())

    # Get raw array values for population and flood
    fld_array = flood_data[0].values
    fld_array[fld_array < min_haz_threshold] = np.nan  # Set values below min threshold to nan
    # fld_array[fld_array > max_haz_threshold] = max_haz_threshold  # Cap large values to maximum threshold value

    # Assign impact factor
    impact_array = damage_factor(fld_array)

    # Create raster from array
    impact_rst = xr.DataArray(np.array([impact_array]).astype(np.float32), coords=flood_data.coords, dims=flood_data.dims)
    # impact_rst.plot()  # to preview

    # Probability of return period
    # Essentially the same as 1/RP, but accounts for cases where RP == 1
    RPp = 1 - np.exp(-1/rp)

    # For each ADM2 region
    for row in result_df.itertuples():
        geom = adm3_data.loc[adm2_data.ADM3_CODE == row.ADM3_CODE, "geometry"]

        # Clip impacted dataset to current ADM2 region
        rst = impact_rst.rio.clip(geom, adm3_data.crs)

        # Determine population in ADM3
        pop_clip = pop_data.rio.clip(geom, adm2_data.crs)
        pop_arr = pop_clip.values
        result_df.loc[result_df.ADM3_CODE == row.ADM3_CODE, "ADM3_Pop"] = pop_arr[(pop_arr > 0, )].sum()

        # Calculate affected population in ADM2
        tmp = pop_arr[(rst > 0, )]
        affected_pop = tmp[tmp > 0].sum()

        # Get Expected Annual Impact for given RP
        RPi_EAI = affected_pop * RPp

        # Save total population affected to dataframe
        result_df.loc[result_df.ADM3_CODE == row.ADM3_CODE, f"RP{rp}_pop_sum"] = affected_pop

        # Save EAI (affected population for a single year)
        result_df.loc[result_df.ADM3_CODE == row.ADM3_CODE, f"RP{rp}_EAI"] = RPi_EAI


# Sum all EAI to get total EAI across all RPs
result_df.loc[:, "Pop_EAI"] = result_df.loc[:, result_df.columns.str.contains('_EAI')].sum(axis=1)

# Calculate Pop_EAI%
result_df.loc[:, "Pop_EAI%"] = result_df.loc[:, "Pop_EAI"] / result_df.loc[:, "ADM3_Pop"]  # Percent affected population per year

# Aggregated to ADM1
agg_func = getattr(np, agg_criteria)
result_df.loc[:, f"ADM1_agg_{agg_criteria}"] = agg_func(result_df.loc[:, "Pop_EAI%"])

# Write table of total population in each class, in each ADM2
result_df.to_csv(f"{country}_functional_example_results.csv", index=False)

# Join result dataframe to ADM2 vector data
result_geom = adm3_data.merge(result_df, on="ADM2_CODE")

# Export geopackage
result_geom.to_file(f"{country}_ADM3.gpkg")


RasterioIOError: X:/Work/GeoTemp/CCDR/Nepal/data_script/cache/WorldPop20_NPL_ppp_UNadj_constrained.tif: No such file or directory